Aim to complete as much of this tutorial on your own *before* coming to the practical session.

Use the practical session to get help for any aspect you do not understand or were unable to complete.

# Classification and Regression 1

Learning objectives
1. Implement PLS, Ridge, and Lasso and Elastic Net regression using the popular python library [sklearn](https://scikit-learn.org/stable/)
2. Visualise the regression results 
3. Explore different metrics to evaluate the model performance in regression settings
4. Investigate the effect of scaling the data on the model performance
5. Apply penalised (logistic) regression for classification 

## Import specific packages and functions

In [ ]:
# IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

from sklearn.preprocessing import StandardScaler, scale
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, GridSearchCV
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression, Lasso, Ridge, ElasticNet # for Lasso and Elastic Net logistic regression
from sklearn.cross_decomposition import PLSRegression # for PLS

## Load in dataset

In this tutorial we will use some plasma metabolomics data to predict BMI

In [ ]:
df = pd.read_csv('../Data-main/diabetes_metabolomics_plasma.csv') #import the data 

### Inspect the data

In [ ]:
df.head() ### 

As you can see this data set in its current format is not suitable for our algorithms. 
In python column indices start from 0, we want to subset only the metabolomics columns for feature scaling.

In [ ]:
# Create feature matrix and target vector
X = df.iloc[:,6:]
y = df['BMI']

In [ ]:
X.head() # Try using the .tail() function -- it is similar to the .head() function but displays the last 5 rows 

In order to test our alogrithms we need to set aside some of the data we have. This is practice for machine learning models. We will use 80% of our data to train our model, and the remaining 20% will be used to test the performance of our model. 

Scikit-Learn has a function ```train_test_split``` to easily do this for us.

In [ ]:
# enter your CID here, or date of birth, or another number of your choosing to use as random state
CID = 0

# remember to check the documentation of each algorithm if setting the random_state is needed
# for this tutorial and all upcoming tutorials...

In [ ]:
# Split the df into 80% train 20% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=CID)

It is crucial that all of the data it is comparing is on the same scale. In our metabolomics data, most of the data is continuous. We will scale the data using the ```StandardScaler()``` shown in the previous tutorials. 

When scaling your data you want to fit the model to your training data, and only transform your testing data. 

In [ ]:
# Normalize Data

# Instantiate scaler model
scaler = StandardScaler()

# Fit and Transform X_train
X_train_scaled = scaler.fit_transform(X_train)

# Transform X_test
X_test_scaled = scaler.transform(X_test)

## PLS Regression 

We will first look at [PLSRegression()](https://scikit-learn.org/stable/modules/generated/sklearn.cross_decomposition.PLSRegression.html).

In [ ]:
plsr = PLSRegression(n_components=1)
plsr.fit(X_train_scaled, y_train)
y_pred = plsr.predict(X_test_scaled)

#### Metrics
Today we will use the following metrics
- explained_variance_score
- r2_score
- mean_squared_error
- mean_absolute_error
- SMAPE (Symmetric Mean Absolute Percentage Error)

In [ ]:
# This is just a helper function to get the smape metric - no need to change 
def smape(A, F):
    out =  100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))
    return(out)

In [ ]:
## Here we will define several helper functions to compute these metrics 
def RegmodelPerformance(y_true, y_pred):
    exp_var = metrics.explained_variance_score(y_true, y_pred)
    r_square = metrics.r2_score(y_true, y_pred)
    MSE = metrics.mean_squared_error(y_true, y_pred)
    MAE = metrics.mean_absolute_error(y_true, y_pred)
    SMAPE = smape(np.array(y_true), y_pred)
    return(exp_var, r_square, MSE, MAE, SMAPE)

def printPerformance(y_true, y_pred):
    exp_var, r_square, MSE, MAE, SMAPE = RegmodelPerformance(y_true, y_pred)
    print("explained variance score = " "%.4f" % exp_var)
    print("R2 = " "%.4f" % r_square)
    print("MSE = " "%.4f" % MSE)
    print("MAE = " "%.4f" % MAE)
    print("SMAPE = " "%.4f" % SMAPE)
    np.set_printoptions(precision=2)

In [ ]:
printPerformance(y_test, y_pred)

Whilst we have trained and fit a PLS model, we have not optimised the number of components. We also have not done any cross validation.

In [ ]:
cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=42)

mse = []
n = len(X_train)

# Calculate MSE using cross-validation, adding one component at a time
for i in np.arange(1, 10):
    pls = PLSRegression(n_components=i)
    score = -1*model_selection.cross_val_score(pls, scale(X_train), y_train, cv=cv,
               scoring='neg_mean_squared_error').mean()
    mse.append(score)

#plot test MSE vs. number of components - lower is better (smaller error)
plt.plot(np.arange(1, 10),mse)
plt.xlabel('Number of PLS Components')
plt.ylabel('MSE')
plt.title('Cross-validation Metrics')


Alternatively, we can also run our own 10-fold CV.

Try below to fill in with your own code to the cross-validation loop

In [ ]:
exp_vars = []
r_squares = []
MSEs = []
MAEs = []
SMAPEs = [] 
from sklearn.model_selection import KFold
kf = KFold(n_splits=2, random_state=CID, shuffle=False)
kf.get_n_splits(X_train_scaled)
for train_index, test_index in kf.split(X_train):
    # add here code to estimate the optimal number of components from the training set
    
    

In [ ]:
# Calculate MSE using cross-validation, 
numSplits = 10
exp_var_mean = 0 
r_square_mean = 0
MSE_mean = 0 
MAE_mean = 0
SMAPE_mean = 0 
kf = KFold(n_splits=numSplits, random_state=CID, shuffle=False)
kf.get_n_splits(X_train)

for train_index, val_index in kf.split(X_train):
    X_traincv = X_train.iloc[train_index]
    X_valcv = X_train.iloc[val_index]
    y_traincv = y_train.iloc[train_index]
    y_valcv = y_train.iloc[val_index]
    X_traincv = scaler.fit_transform(X_traincv)
    X_valcv = scaler.transform(X_valcv)
    pls = PLSRegression(n_components=2) ## i is the no. components, consider 
    pls.fit(X_traincv, y_traincv)
    ypred = pls.predict(X_valcv)
    exp_var, r_square, MSE, MAE, SMAPE = RegmodelPerformance(y_valcv, ypred)
    exp_var_mean+= exp_var
    r_square_mean += r_square
    MSE_mean += MSE
    MAE_mean += MAE
    SMAPE_mean += SMAPE
    
    

In [ ]:
## in the space below compute the mean and standard deviations of your metrics
## I have done the first one as an example

print(exp_var_mean/numSplits)
print(r_square_mean/numSplits)

Now have a think how you might do this and optimise the number of components at the same time

In [ ]:
exp_vars = []
r_squares = []
MSEs = []
MAEs = []
SMAPEs = [] 
# Calculate MSE using cross-validation, adding one component at a time
for i in range(1, 12):   #Here we are defining how many components 
    exp_var_mean = 0 
    r_square_mean = 0
    MSE_mean = 0 
    MAE_mean = 0
    SMAPE_mean = 0 
    kf = KFold(n_splits=10, random_state=None, shuffle=False) ## can also input and value you like for random state but also set shuffle = Ture
    kf.get_n_splits(X_train)

    for train_index, val_index in kf.split(X_train):
        X_traincv = X_train.iloc[train_index]
        X_valcv = X_train.iloc[val_index]
        y_traincv = y_train.iloc[train_index]
        y_valcv = y_train.iloc[val_index]
        X_traincv = scaler.fit_transform(X_traincv)
        X_valcv = scaler.fit_transform(X_valcv)
        pls = PLSRegression(n_components=i) ## i is the no. components
        pls.fit(X_traincv, y_traincv)
        ypred = pls.predict(X_valcv)
        exp_var, r_square, MSE, MAE, SMAPE = RegmodelPerformance(y_valcv, ypred)
        exp_var_mean+= exp_var
        r_square_mean += r_square
        MSE_mean += MSE
        MAE_mean += MAE
        SMAPE_mean += SMAPE
    exp_vars.append(exp_var_mean/10)
    r_squares.append(r_square_mean/10)
    MSEs.append(MSE_mean/10)
    MAEs.append(MAE_mean/10)
    SMAPEs.append(SMAPE_mean/10)

In [ ]:
len(MSEs) ## printing len MSE should be the same as however many components you have looked into 

In [ ]:
#plot test MSE vs. number of components
plt.plot(range(1,12),MSEs)
plt.xlabel('Number of PLS Components')
plt.ylabel('MSE')
plt.title('Cross-validation Metrics')


Results vary slightly depending on the split seed values, although it looks like the best model is the one with 4 components? Do you get the same result? Do you want to use a different metric than MSE to find the optimum?

In [ ]:
n_comp_opt = np.array(MSEs).argsort()[0]+1 # finding the index of the lowest value
plsbest = PLSRegression(n_components=n_comp_opt)
plsbest.fit(X_train_scaled, y_train)
y_pred = plsbest.predict(X_test_scaled)
printPerformance(y_test, y_pred)

### Feature importance

What are the most important features in the model? Inspect the ```coef_``` attribute and see which variables have the biggest (absolute) weights (loadings) in the model.

In [ ]:
# get feature coefficients
coef = plsbest.coef_
# get feature names
cnames = list(X.columns)
# sort these
_, coef, cnames = zip(*sorted(zip(abs(coef), coef, cnames),reverse=True))
coef = np.concatenate(coef)
# show top 10 features
nfeat = 10
ind = [*range(nfeat, 0, -1)]
cp = coef[0:nfeat]
ind = np.asarray(ind)
# plotting the results
fig, ax = plt.subplots()
ax.barh(ind,cp)
ax.set_yticks(ind, labels=cnames[0:nfeat])
ax.set_xlabel('PLS loading weights')

## Lasso 

Now we will use the [Lasso()](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) function. We will optimise the model parameters using a [GridSearch](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

In [ ]:
# define model
model = Lasso()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=CID)
# define grid
grid = dict()
grid['alpha'] = np.arange(0, 1, 0.01)
# define search
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(X_train_scaled, y_train)
# summarize
print('MAE: %.3f' % (results.best_score_*-1))
print('Config: %s' % results.best_params_)

Predict the test set now, how does this compare with PLS?

In [ ]:
y_pred = search.predict(X_test_scaled)
printPerformance(y_test, y_pred)

### Feature importance

What are the most important features in the model? Inspect the ```coef_``` attribute and see which variables have non-zero weights in the model. Note that the GridSearchCV model does not have this attribute, so we must get it another way...

In [ ]:
# get the best parameters
best_param = results.best_params_

# fit a new model with these parameters
lasso = Lasso(alpha=results.best_params_["alpha"])
lasso.fit(X_train_scaled, y_train)

In [ ]:
# get feature coefficients
coef = lasso.coef_
# get feature names
cnames = list(X.columns)
# sort these
acoef, coef, cnames = zip(*sorted(zip(abs(coef), coef, cnames),reverse=True))
coef = np.asarray(coef)
nonzerofeat = sum(np.asarray(acoef)>0)
ind = np.asarray([*range(nonzerofeat, 0, -1)])
# plotting the results
fig, ax = plt.subplots(figsize=(10,15))
ax.barh(ind,coef[0:nonzerofeat])
ax.set_yticks(ind, labels=cnames[0:nonzerofeat])
ax.set_xlabel('Lasso coefficients')

## Ridge 

Try right the same as the above but now using the [Ridge()](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) function.

In [ ]:
# define model and try the rest yourself
model = Ridge()

In [ ]:
# define model
model = Ridge()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=CID)
# define grid
grid = dict()
grid['alpha'] = np.arange(0, 1, 0.01)
# define search
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(X_train_scaled, y_train)
# summarize
print('MAE: %.3f' % (results.best_score_*-1))
print('Config: %s' % results.best_params_)

In [ ]:
# test set performane 
y_pred = search.predict(X_test_scaled)
printPerformance(y_test, y_pred)

### Feature importance

What are the most important features in the model? Inspect the ```coef_``` attribute similarly to Lasso by creating a new model, and plotting the top n features as with PLS.

In [ ]:
# get the best parameters
best_param = results.best_params_

# fit a new model with these parameters
ridge = Ridge(alpha=results.best_params_["alpha"])
ridge.fit(X_train_scaled, y_train)

In [ ]:
# get feature coefficients
coef = ridge.coef_
# get feature names
cnames = list(X.columns)
# sort these
_, coef, cnames = zip(*sorted(zip(abs(coef), coef, cnames),reverse=True))
coef = np.asarray(coef)
# show top 10 features
nfeat = 10
ind = [*range(nfeat, 0, -1)]
cp = coef[0:nfeat]
ind = np.asarray(ind)
# plotting the results
fig, ax = plt.subplots()
ax.barh(ind,cp)
ax.set_yticks(ind, labels=cnames[0:nfeat])
ax.set_xlabel('Ridge regression coefficients')

### Compare the Ridge results with Lasso and PLS, which model is most predictive, which are the important features?

## Elastic-Net 

Elastic Net is a mix of both ridge and lasso, where we can tune both of the parameters.

Don't worry if the code below (using the [ElasticNet()](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) function) returns some warnings around the model not converging. However, __do think about what this means...__ (could you change some parameters?)

In [ ]:
# define model
model = ElasticNet()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=CID)
# define grid
grid = dict()
grid = {"max_iter": [10, 50, 100],
                      "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                      "l1_ratio": np.arange(0.0, 1.0, 0.1)}
# define search
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(X_train_scaled, y_train)
# summarize
print('MAE: %.3f' % (results.best_score_*-1))
print('Config: %s' % results.best_params_)

In [ ]:
# test set performane 
y_pred = search.predict(X_test_scaled)
printPerformance(y_test, y_pred)

### Feature importance

What are the most important features in the model? Inspect the ```coef_``` attribute and see which variables have non-zero weights in the model as with the Lasso model.

In [ ]:
# get the best parameters
best_param = results.best_params_

# fit a new model with these parameters
enet = ElasticNet(alpha=results.best_params_["alpha"], l1_ratio=results.best_params_["l1_ratio"], max_iter=results.best_params_["max_iter"])
enet.fit(X_train_scaled, y_train)

In [ ]:
# get feature coefficients
coef = enet.coef_
# get feature names
cnames = list(X.columns)
# sort these
acoef, coef, cnames = zip(*sorted(zip(abs(coef), coef, cnames),reverse=True))
coef = np.asarray(coef)
nfeat = sum(np.asarray(acoef)>0)
if nfeat > 100:
    nfeat = 50 # limit number of features shown, can increase/decrease these numbers
ind = np.asarray([*range(nfeat, 0, -1)])
# plotting the results
fig, ax = plt.subplots(figsize=(10,15))
ax.barh(ind,coef[0:nfeat])
ax.set_yticks(ind, labels=cnames[0:nfeat])
ax.set_xlabel('Elastic Net coefficients')

### What's Next?
We have walked through how to implement PLS, Ridge, Lasso and Elastic Net regression.

For further understanding and practice:
- Try using a different scaling to see how it affects results: ```robust_scale``` / ```RobustScaler```, ```power_transform``` / ```PowerTransformer```
- Change certain paramaters such as number of folds, ncomponents (PLS), alpha and lambda values (Elastic net, Lasso, Ridge), e.g try a larger grid search 
- Use a different dataset for a regression problem
- Implement different error metrics for regression
- Implement the classification (logistic regression) versions of these, look at examples here for [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifier.html) and [Lasso and ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), and you can use a binary vector as input to [PLSRegression](https://scikit-learn.org/stable/modules/generated/sklearn.cross_decomposition.PLSRegression.html).

## Your turn
Select another dataset from the `Data` folder, import and inspect it.

Is there a continuous outcome you can use for regression?

If yes, use one of the methods above to model it.

If not, try to implement the classification versions of these algorithms.

How well do the methods perform on the new dataset? Does the same method as above perform best?

In [ ]:
# Import datasets...

In [ ]:
# Split data and perform scaling...

In [ ]:
# Calculate a regression or classification model...

In [ ]:
# Find the important features